### Running PESTPP-IES

In [ ]:
import os
import shutil
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt;
import psutil

import sys
import pyemu
import flopy
assert "dependencies" in flopy.__file__
assert "dependencies" in pyemu.__file__
sys.path.insert(0,"..")
import herebedragons as hbd

In [ ]:
# specify the temporary working folder
t_d = os.path.join('pst_template_ies')
# get the previously generated PEST dataset
org_t_d = os.path.join("pst_template_pmc")
if not os.path.exists(org_t_d):
    raise Exception()
if os.path.exists(t_d):
    shutil.rmtree(t_d)
shutil.copytree(org_t_d,t_d)

In [ ]:
pst_path = os.path.join(t_d, 'pest.pst')
pst = pyemu.Pst(pst_path)

In [ ]:
# check to see if obs&weights notebook has been run
if not pst.nnz_obs > 0:
    raise Exception()

In [ ]:
pst.pestpp_options["ies_num_reals"] = 100
pst.pestpp_options["ies_bad_phi_sigma"] = 1.75
pst.pestpp_options["ies_multimodal_alpha"] = 0.99

In [ ]:
pst.control_data.noptmax = -2
pst.write(os.path.join(t_d, 'pest.pst'),version=2)

pyemu.os_utils.run("pestpp-ies pest.pst",cwd=t_d)

In [ ]:
with open(os.path.join(t_d,"phi.csv"),'w') as f:
    f.write("hds,0.5\n")
    f.write("drn,0.5\n")
pst.pestpp_options["ies_phi_factor_file"] = "phi.csv"

In [ ]:
pst.control_data.noptmax = -2
pst.write(os.path.join(t_d, 'pest.pst'),version=2)

pyemu.os_utils.run("pestpp-ies pest.pst",cwd=t_d)

In [ ]:
pst.control_data.noptmax = 5
pst.write(os.path.join(t_d, 'pest.pst'),version=2)

In [ ]:
num_workers = 20
m_d = "master_ies0"

In [ ]:
pyemu.os_utils.start_workers(t_d, # the folder which contains the "template" PEST dataset
                            'pestpp-ies', #the PEST software version we want to run
                            'pest.pst', # the control file to use with PEST
                            num_workers=num_workers, #how many agents to deploy
                            worker_root='.', #where to deploy the agent directories; relative to where python is running
                            master_dir=m_d, #the manager directory
                            )

In [ ]:
pst = pyemu.Pst(os.path.join(m_d,"pest.pst"))
obs = pst.observation_data
oe_pr = pst.ies.obsen0
oe_pt = pst.ies.get("obsen",pst.ies.phiactual.iteration.max())
forecasts = pst.pestpp_options["forecasts"].split(",")
for f in forecasts:
    ax = oe_pr.loc[:,f].plot(kind="hist",fc="0.5",alpha=0.5,density=True)
    ax = oe_pt.loc[:,f].plot(kind="hist",fc="b",alpha=0.5,density=True)
    ylim = ax.get_ylim()
    v = obs.loc[f,"obsval"]
    ax.plot([v,v],ylim,"r-",lw=2)
    ax.set_title(f)
    plt.show()
        